# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [83]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

In [84]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")
include("../../../src/functions/jacobi_field/beta.jl")

β (generic function with 1 method)

In [85]:
black = RGBA{Float64}(colorant"#000000")
TolVibrantOrange = RGBA{Float64}(colorant"#EE7733")
TolVibrantBlue = RGBA{Float64}(colorant"#0077BB")
TolVibrantTeal = RGBA{Float64}(colorant"#009988")
TolVibrantMagenta = RGBA{Float64}(colorant"#EE3377")
TolVibrantCyan = RGBA{Float64}(colorant"#33BBEE");
render_size = 1

1

### Load data and construct manifold ###

In [86]:
# load data
M = Sphere(2)
d = manifold_dimension(M)
n = 100  # 100


100

In [87]:
θ = collect(range(0, 2 * π, n+1))[1:n]
predata = [[cos(θ[i]), sin(θ[i]), 0.] for i in 1:n]
σ = .05  # variance

Random.seed!(31)
data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ S(2)^n

In [88]:
# Export slice image
asymptote_export_S2_signals("results/artificial1D_orig.asy", points=[data],
colors=Dict(:points => [TolVibrantBlue]),
dot_size=3.5,
camera_position=(1.0, 0.5, 0.5)); 

### Construct low rank approximation ###

In [89]:
q₁ = mean(M, data)
println(q₁)
log_q_data₁ = log.(Ref(M), Ref(q₁), data);  # ∈ T_q P(3)^n
# TODO compute theoretical lower bound
ONB₁ = get_basis.(Ref(M), Ref(q₁), DiagonalizingOrthonormalBasis.(log_q_data₁))
κ₁ = maximum([maximum(distance(M, q₁, data[i])^2 .* ONB₁[i].data.eigenvalues) for i in 1:n])
println(κ₁)


summed_pairwise_distances = [sum([distance.(Ref(M), Ref(data[i]), data) .^2]) for i in 1:n]
q₂ = data[argmin(summed_pairwise_distances)]
log_q_data₂ = log.(Ref(M), Ref(q₂), data);  # ∈ T_q P(3)^n
println(q₂)
ONB₂ = get_basis.(Ref(M), Ref(q₂), DiagonalizingOrthonormalBasis.(log_q_data₂))
κ₂ = maximum([maximum(distance(M, q₂, data[i])^2 .* ONB₂[i].data.eigenvalues) for i in 1:n])
println(κ₂)

[0.005864527816832297, -0.004603373218850059, 0.9999722077480426]
2.826077714333991
[0.9973810846474874, -0.05040543956910714, 0.05186775153451892]
9.167247539594285


In [90]:
curvature_corrected_low_rank_approximation(M, q₁, data, 1; stepsize=1/10, max_iter=10); 

Initial  F(x): 0.42256497554 | 
# 1     change: 0.328193618 |  F(x): 0.41857374134 | 
# 2     change: 0.272227296 |  F(x): 0.41582767158 | 
# 3     change: 0.225806336 |  F(x): 0.41393828015 | 
# 4     change: 0.187302654 |  F(x): 0.41263829244 | 
# 5     change: 0.155365861 |  F(x): 0.41174382505 | 
# 6     change: 0.128875895 |  F(x): 0.41112836632 | 
# 7     change: 0.106903738 |  F(x): 0.41070487560 | 
# 8     change: 0.088678829 |  F(x): 0.41041346794 | 
# 9     change: 0.073562037 |  F(x): 0.41021294136 | 
# 10    change: 0.061023244 |  F(x): 0.41007494776 | 
The algorithm reached its maximal number of iterations (10).


In [91]:
curvature_corrected_low_rank_approximation(M, q₂, data, 1; stepsize=1/10, max_iter=10); 

Initial  F(x): 0.01776199510 | 
# 1     change: 0.161369540 |  F(x): 0.01704129874 | 
# 2     change: 0.130582351 |  F(x): 0.01656936792 | 
# 3     change: 0.105669058 |  F(x): 0.01626033461 | 
# 4     change: 0.085508994 |  F(x): 0.01605797046 | 
# 5     change: 0.069195302 |  F(x): 0.01592545588 | 
# 6     change: 0.055994145 |  F(x): 0.01583868053 | 
# 7     change: 0.045311694 |  F(x): 0.01578185641 | 
# 8     change: 0.036667418 |  F(x): 0.01574464517 | 
# 9     change: 0.029672468 |  F(x): 0.01572027701 | 
# 10    change: 0.024012182 |  F(x): 0.01570431891 | 
The algorithm reached its maximal number of iterations (10).


In [92]:
exact_low_rank_approximation(M, q₁, data, 1; stepsize=1/20, max_iter=20); 

Initial  F(x): 0.40332784783 | 
# 1     change: 0.144278613 |  F(x): 0.40123815824 | 
# 2     change: 0.140723757 |  F(x): 0.39934230430 | 
# 3     change: 0.137650349 |  F(x): 0.39762392079 | 
# 4     change: 0.135021760 |  F(x): 0.39606914241 | 
# 5     change: 0.132807322 |  F(x): 0.39466627665 | 
# 6     change: 0.130981608 |  F(x): 0.39340553931 | 
# 7     change: 0.129523835 |  F(x): 0.39227884187 | 
# 8     change: 0.128417393 |  F(x): 0.39127962256 | 
# 9     change: 0.127649472 |  F(x): 0.39040271459 | 
# 10    change: 0.127210768 |  F(x): 0.38964424675 | 
# 11    change: 0.127095274 |  F(x): 0.38900157278 | 
# 12    change: 0.127300132 |  F(x): 0.38847322697 | 
# 13    change: 0.127825546 |  F(x): 0.38805890452 | 
# 14    change: 0.128674762 |  F(x): 0.38775946594 | 
# 15    change: 0.129854097 |  F(x): 0.38757696546 | 
# 16    change: 0.131373026 |  F(x): 0.38751470463 | 
# 17    change: 0.133244331 |  F(x): 0.38757731269 | 
# 18    change: 0.135484314 |  F(x): 0.38777085653

In [93]:
exact_low_rank_approximation(M, q₂, data, 1; stepsize=1/100, max_iter=20); 

Initial  F(x): 0.00720070851 | 
# 1     change: 0.005902608 |  F(x): 0.00721576263 | 
# 2     change: 0.005939454 |  F(x): 0.00723115553 | 
# 3     change: 0.005976841 |  F(x): 0.00724689301 | 
# 4     change: 0.006014772 |  F(x): 0.00726298095 | 
# 5     change: 0.006053245 |  F(x): 0.00727942535 | 
# 6     change: 0.006092261 |  F(x): 0.00729623230 | 
# 7     change: 0.006131820 |  F(x): 0.00731340798 | 
# 8     change: 0.006171922 |  F(x): 0.00733095870 | 
# 9     change: 0.006212567 |  F(x): 0.00734889086 | 
# 10    change: 0.006253755 |  F(x): 0.00736721095 | 
# 11    change: 0.006295485 |  F(x): 0.00738592559 | 
# 12    change: 0.006337758 |  F(x): 0.00740504150 | 
# 13    change: 0.006380572 |  F(x): 0.00742456550 | 
# 14    change: 0.006423927 |  F(x): 0.00744450454 | 
# 15    change: 0.006467823 |  F(x): 0.00746486567 | 
# 16    change: 0.006512260 |  F(x): 0.00748565604 | 
# 17    change: 0.006557236 |  F(x): 0.00750688294 | 
# 18    change: 0.006602750 |  F(x): 0.00752855375

In [94]:
max_iter = 50

nR_q₁ = []
nU₁ = []
ccR_q₁ = []
ccU₁ = []
eR_q₁ = []
eU₁ = []

nR_q₂ = []
nU₂ = []
ccR_q₂ = []
ccU₂ = []
eR_q₂ = []
eU₂ = []

for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q₁, nUr₁ = naive_low_rank_approximation(M, q₁, data, i)
    push!(nR_q₁, nRr_q₁)
    push!(nU₁, nUr₁)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₁, ccUr₁ = curvature_corrected_low_rank_approximation(M, q₁, data, i; stepsize=1/10, max_iter=max_iter); 
    push!(ccR_q₁, ccRr_q₁)
    push!(ccU₁, ccUr₁)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q₁, eUr₁ = exact_low_rank_approximation(M, q₁, data, i; stepsize=1/50, max_iter=max_iter); 
    push!(eR_q₁, eRr_q₁)
    push!(eU₁, eUr₁)

    println("#$(i) | computing naive low-rank approximation")
    nRr_q₂, nUr₂ = naive_low_rank_approximation(M, q₂, data, i)
    push!(nR_q₂, nRr_q₂)
    push!(nU₂, nUr₂)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₂, ccUr₂ = curvature_corrected_low_rank_approximation(M, q₂, data, i; stepsize=1/10, max_iter=max_iter); 
    push!(ccR_q₂, ccRr_q₂)
    push!(ccU₂, ccUr₂)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q₂, eUr₂ = exact_low_rank_approximation(M, q₂, data, i; stepsize=1/100, max_iter=max_iter); 
    push!(eR_q₂, eRr_q₂)
    push!(eU₂, eUr₂)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
Initial  F(x): 0.42256497554 | 
# 1     change: 0.328193618 |  F(x): 0.41857374134 | 
# 2     change: 0.272227296 |  F(x): 0.41582767158 | 
# 3     change: 0.225806336 |  F(x): 0.41393828015 | 
# 4     change: 0.187302654 |  F(x): 0.41263829244 | 
# 5     change: 0.155365861 |  F(x): 0.41174382505 | 
# 6     change: 0.128875895 |  F(x): 0.41112836632 | 
# 7     change: 0.106903738 |  F(x): 0.41070487560 | 
# 8     change: 0.088678829 |  F(x): 0.41041346794 | 
# 9     change: 0.073562037 |  F(x): 0.41021294136 | 
# 10    change: 0.061023244 |  F(x): 0.41007494776 | 
# 11    change: 0.050622751 |  F(x): 0.40997998260 | 
# 12    change: 0.041995855 |  F(x): 0.40991462579 | 
# 13    change: 0.034840059 |  F(x): 0.40986964349 | 
# 14    change: 0.028904460 |  F(x): 0.40983868210 | 
# 15    change: 0.023980949 |  F(x): 0.40981736976 | 
# 16    change: 0.019896915 |  F(x): 0.40980269811 | 
# 

In [95]:
# basepoint q₁
ref_distance₁ = sum(distance.(Ref(M), Ref(q₁), data).^2)

naive_tangent_distances_r₁ = zeros(d)
predicted_naive_distances_r₁ = zeros(d)
true_naive_distances_r₁ = zeros(d)

curvature_corrected_tangent_distances_r₁ = zeros(d)
predicted_curvature_corrected_distances_r₁ = zeros(d)
true_curvature_corrected_distances_r₁ = zeros(d)

exact_tangent_distances_r₁ = zeros(d)
exact_distances_r₁ = zeros(d)

# basepoint q₂
ref_distance₂ = sum(distance.(Ref(M), Ref(q₂), data).^2)

naive_tangent_distances_r₂ = zeros(d)
predicted_naive_distances_r₂ = zeros(d)
true_naive_distances_r₂ = zeros(d)

curvature_corrected_tangent_distances_r₂ = zeros(d)
predicted_curvature_corrected_distances_r₂ = zeros(d)
true_curvature_corrected_distances_r₂ = zeros(d)

exact_tangent_distances_r₂ = zeros(d)
exact_distances_r₂ = zeros(d)

for rank in 1:d
    ## basepoint q₁
    naive_log_q_data_r₁ = [sum([nR_q₁[rank][i] * nU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₁ = [sum([ccR_q₁[rank][i] * ccU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₁ = [sum([eR_q₁[rank][i] * eU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # expoentiate back
    naive_data_r₁ = exp.(Ref(M), Ref(q₁), naive_log_q_data_r₁)
    curvature_corrected_data_r₁ = exp.(Ref(M), Ref(q₁), curvature_corrected_log_q_data_r₁)
    exact_data_r₁ = exp.(Ref(M), Ref(q₁), exact_log_q_data_r₁)


    # compute relative tangent space error
    naive_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - naive_log_q_data_r₁).^2) / ref_distance₁
    curvature_corrected_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - curvature_corrected_log_q_data_r₁).^2) / ref_distance₁
    exact_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - exact_log_q_data_r₁).^2) / ref_distance₁


    # compute relative manifold error
    predicted_naive_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, naive_log_q_data_r₁)
    true_naive_distances_r₁[rank] = exact_loss(M, q₁, data, naive_log_q_data_r₁)
    predicted_curvature_corrected_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    true_curvature_corrected_distances_r₁[rank] = exact_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    exact_distances_r₁[rank] = exact_loss(M, q₁, data, exact_log_q_data_r₁)

    ## basepoint q₂
    naive_log_q_data_r₂ = [sum([nR_q₂[rank][i] * nU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₂ = [sum([ccR_q₂[rank][i] * ccU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₂ = [sum([eR_q₂[rank][i] * eU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # expoentiate back
    naive_data_r₂ = exp.(Ref(M), Ref(q₂), naive_log_q_data_r₂)
    curvature_corrected_data_r₂ = exp.(Ref(M), Ref(q₂), curvature_corrected_log_q_data_r₂)
    exact_data_r₂ = exp.(Ref(M), Ref(q₂), exact_log_q_data_r₂)


    # compute relative tangent space error
    naive_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - naive_log_q_data_r₂).^2) / ref_distance₂
    curvature_corrected_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - curvature_corrected_log_q_data_r₂).^2) / ref_distance₂
    exact_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - exact_log_q_data_r₂).^2) / ref_distance₂


    # compute relative manifold error
    predicted_naive_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, naive_log_q_data_r₂)
    true_naive_distances_r₂[rank] = exact_loss(M, q₂, data, naive_log_q_data_r₂)
    predicted_curvature_corrected_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    true_curvature_corrected_distances_r₂[rank] = exact_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    exact_distances_r₂[rank] = exact_loss(M, q₂, data, exact_log_q_data_r₂)
end

In [96]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
# basepoint q₁
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁, true_naive_distances_r₁, true_curvature_corrected_distances_r₁, exact_distances_r₁], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_1.png")
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁ .+ 1e-16, true_naive_distances_r₁ .+ 1e-16, true_curvature_corrected_distances_r₁ .+ 1e-16, exact_distances_r₁ .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_1.png")
# basepoint q₂
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂, true_naive_distances_r₂, true_curvature_corrected_distances_r₂, exact_distances_r₂], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_2.png")
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂ .+ 1e-16, true_naive_distances_r₂ .+ 1e-16, true_curvature_corrected_distances_r₂ .+ 1e-16, exact_distances_r₂ .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_2.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_logerrors_by_rank_2.png"

In [97]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
# basepoint q₁
plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_1.png")
plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank_1.png")
# basepoint q₂
plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_2.png")
plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank_2.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_logdiscrepancy_by_rank_2.png"